In [1]:
import pandas as pd 
import numpy as np
import os
from openpyxl import load_workbook
from os import listdir
from os.path import isfile, join
import re

In [2]:
def filter_type(df,col,type):
    data=df.loc[df[col]==type]
    return data

def filter_list(df,col,type_list):
    data=df.loc[df[col].isin(type_list)]
    return data

def todate(df):
    df['team_report_create_date']=df['team_report_create_date'].astype('datetime64')
    return df
def activeusers(df):
    num=df.groupby(by=[df.index.year,df.index.month])['team_report_user_id'].nunique()
    return num

def quantiles_df(df,segment,mon):
    benchmark=pd.DataFrame(columns=['month','segment','25th','50th','75th','90th','mean','total users','total candidates'])
    #get monthly sum for every user
    grp=df.groupby(by='team_report_user_id').sum()
    #ger number of unique users every mon
    num_of_users=len(df['team_report_user_id'].drop_duplicates())
    try:
        nlist=grp
        total=nlist.sum().values
        mean=nlist.mean().values
        quantiles=nlist.quantile([0.25,0.5,0.75,0.9]).values.tolist()
        benchmark.loc[0,'month']='2020/'+str(mon)
        benchmark.loc[0,'segment']=segment
        benchmark.loc[0,2:6]=quantiles
        benchmark.loc[0,'mean']=mean
        benchmark.loc[0,'total users']=num_of_users
        benchmark.loc[0,'total candidates']=total
    except:
        benchmark.loc[0,'month']='2020/'+str(mon)
        benchmark.loc[0,'segment']=segment
        benchmark.loc[2:7]='nan'
    return benchmark


def getall(df,segments,acc_grp,mon,seg_num):
    benchmark=pd.DataFrame(columns=['month','segment','25th','50th','75th','90th','mean','total users','total candidates'])
    for i in range(seg_num):
        segment=segments[i]
        acc_list=acc_grp.loc[(segment)][0].tolist()
        data=df.loc[df['team_report_team_id'].isin(acc_list)]
        df_part=quantiles_df(data,segment,mon)
        benchmark=benchmark.append(df_part)
    df_overall=quantiles_df(df,'Overall',mon)
    benchmark=benchmark.append(df_overall)
    return benchmark

def get_active_days (df,segments,acc_grp,num):
    benchmark=pd.DataFrame(columns=['month','segment','25th','50th','75th','90th','mean','total users','total candidates'])    

In [4]:
path="C:/3-实习/Hiretual/数据分析/user_benchmark/data/"
acc_mapping="acc_mapping_dec.csv" #from salesforce
raw_data="benchmark_data_dec.csv" # from sql
email_data="email_data_dec.csv" #from sql
output="test.xlsx" 
output_path = path+output
mon=1
seg_num=11

###1 account mapping - this is to group accounts into different segments 
accounts=pd.read_csv(path+acc_mapping)
acc_grp=accounts.groupby(by=[accounts['Industry'],accounts['Market Segment']])['BMC_ID__c'].unique().reset_index()
industry_list=['Professional Services','Healthcare','IT & Software','Staffing & Recruiting']
acc_grp=acc_grp.loc[acc_grp['Industry'].isin(industry_list)]
acc_grp=acc_grp.set_index(['Industry','Market Segment'])
segments=acc_grp.index.to_list()
#reorder
#order = [6,7,8,0,1,2,3,4,5,9,10,11]
order = [6,7,0,1,2,3,4,5,8,9,10]
segments = [segments[i] for i in order]
print('acc_mapping done')

###2 get raw data
data=pd.read_csv(path+raw_data)
df=pd.pivot_table(data, values='team_report_value', index=['team_subscription_poc_company', 'team_report_team_id','team_report_user_id'],
columns=['team_report_type'], aggfunc=np.sum)
df['cf_rate']=df['talents_with_contact']/df['total_sourced']
df['q_rate']=df['total_qualified']/df['user_uniq_viewed_talent']
df=df.reset_index()

email_data=pd.read_csv(path+email_data)
email_data=email_data.rename(columns={"email_thread_team_id": "team_report_team_id", "email_thread_user_id": "team_report_user_id","email_sent":"candidates_contacted"})
email_data['reply_rate']=email_data['reply_candidates']/email_data['candidates_contacted']
email_data['open_rate']=email_data['open_candidates']/email_data['candidates_contacted']

df = pd.merge(df,email_data, how='left',on=['team_report_team_id','team_report_user_id'])
df=df.set_index(['team_report_team_id','team_report_user_id'])
types=list(df.columns)
print('raw_data cleaning done')

###3. read to new csv
book = load_workbook(output_path)
writer = pd.ExcelWriter(output_path, engine = 'openpyxl')


for type_name in types:
    df_type=df[type_name]
    df_type=df_type.reset_index()
    name = type_name
    seg_table=getall(df_type,segments,acc_grp,mon,seg_num)
    table=seg_table.sort_values(by=['month','segment'],ascending=[False,True])
    data=table.set_index(['month','segment'])
    #rearrange order
    data=data.reset_index()
    col_list=['25th','50th','75th','90th','mean','total candidates']
    for col in col_list:
        data[col]=data[col].str.get(0)
    data=data.reindex([6,7,0,1,2,3,4,5,8,9,10,11])
    writer.book = book
    data.to_excel(writer, sheet_name = name)

writer.save()
writer.close()
print('file saved') 

acc_mapping done
raw_data cleaning done
file saved
